In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf

from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
print(train_dataset.shape)

(50000, 28, 28)


In [4]:
train_dataset=train_dataset.reshape(train_dataset.shape[0],-1).T
train_labels=train_labels.reshape(train_labels.shape[0],-1).T
test_dataset=train_dataset.reshape(test_dataset.shape[0],-1).T
test_labels=train_labels.reshape(test_labels.shape[0],-1).T

In [5]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(train_labels,10)
#rint(Train_labels.shape)
Train_labels=Train_labels[0]
print(Train_labels.shape)
#rint(train_labels[0])
#rint(Train_labels[0][0])
print(train_dataset.shape)

(50000, 10)
(784, 50000)


In [7]:
total_layers=5
Layer_Units={0:5,1:7,2:8,3:7,4:10}
parameters={}
for i in Layer_Units:
    num=i+1
    weight="W"+str(num)
    bias='b'+str(num)
    if i==0:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],784],initializer=tf.contrib.layers.xavier_initializer(seed=1))
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())
    else:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],Layer_Units[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=1)) 
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())

In [9]:
a=0
z=0
for i in Layer_Units:
        num=i+1
        if i==0:
            z=tf.add(tf.matmul(parameters['W'+str(num)],train_dataset),parameters['b'+str(num)])
            a=tf.nn.relu(z)   
        elif i==total_layers-1:
            z=tf.add(tf.matmul(parameters['W'+str(num)],a),parameters['b'+str(num)])
        else:
            z=tf.add(tf.matmul(parameters['W'+str(num)],a),parameters['b'+str(num)])
            a=tf.nn.relu(z)

In [11]:
logits = tf.transpose(z)
labels = Train_labels
print(logits.shape)
print(labels.shape)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =labels))
print(cost)

(50000, 10)
(50000, 10)
Tensor("Mean:0", shape=(), dtype=float32)
